In [9]:
import numpy as np
from scipy.stats import norm
from typing import Callable, Tuple, Dict

"""============================================================================
Analytic‑Hessian bandwidth selection for univariate KDE
============================================================================
Benchmarks (per kernel) the *same* finite‑sample Least‑Squares‑CV (LSCV)
surface with four optimisers:
    • grid search      –   50 log‑spaced points (reference optimum)
    • golden section   –   derivative‑free line search
    • Newton–Armijo    –   analytic grad & Hess (this work)
    • Silverman        –   closed‑form plug‑in rule

Two kernels are supported – **Gaussian** (classic) and **Epanechnikov** – with
closed‑form convolution, gradient and Hessian for both.
Outputs mean ± s.d. Integrated‑Squared‑Error (ISE) and evaluation counts across
*R* Monte‑Carlo replicates.
============================================================================"""

# ---------------------------------------------------------------------------
# 1.  Kernel primitives & helper polynomials
# ---------------------------------------------------------------------------
SQRT_2PI = np.sqrt(2 * np.pi)
_pairwise_sq = lambda x: (x[:, None] - x[None, :]) ** 2   # n×n squared dist

# Helper that robustly handles a scalar *or* array expression ----------------

def _poly_mask(u: np.ndarray, mask: np.ndarray, expr):
    """Return `expr` on `mask`, 0 elsewhere. `expr` may be scalar or array."""
    out = np.zeros_like(u, dtype=float)
    if np.isscalar(expr):
        out[mask] = expr
    else:  # array‑valued polynomial already evaluated element‑wise
        out[mask] = expr[mask]
    return out

# —— Gaussian ————————————————————————————————————————————
K_gauss      = lambda u: np.exp(-0.5 * u * u) / SQRT_2PI
K_gauss_p    = lambda u: -u * K_gauss(u)                   # derivative
K_gauss_pp   = lambda u: (u * u - 1) * K_gauss(u)          # 2nd‑derivative
K2_gauss     = lambda u: np.exp(-0.25 * u * u) / np.sqrt(4 * np.pi)  # conv.
K2_gauss_p   = lambda u: -0.5 * u * K2_gauss(u)
K2_gauss_pp  = lambda u: (0.25 * u * u - 0.5) * K2_gauss(u)

# —— Epanechnikov —————————————————————————————————————————
#   K(u) = 0.75 (1-u²) 1{|u|≤1};   convolution & derivatives are piecewise
_absu = lambda u: np.abs(u)
K_epan = lambda u: _poly_mask(u, _absu(u) <= 1, 0.75 * (1 - u * u))
K_epan_p  = lambda u: _poly_mask(u, _absu(u) <= 1, -1.5 * u)
K_epan_pp = lambda u: _poly_mask(u, _absu(u) <= 1, -1.5)

# Convolution K*K  (valid for |u|≤2)
K2_epan = lambda u: _poly_mask(
    u,
    _absu(u) <= 2,
    0.6 - 0.75 * _absu(u) ** 2 + 0.375 * _absu(u) ** 3 - 0.01875 * _absu(u) ** 5,
)
K2_epan_p = lambda u: _poly_mask(
    u,
    _absu(u) <= 2,
    np.sign(u) * (-0.09375 * _absu(u) ** 4 + 1.125 * _absu(u) ** 2 - 1.5 * _absu(u)),
)
K2_epan_pp = lambda u: _poly_mask(
    u,
    _absu(u) <= 2,
    -0.375 * _absu(u) ** 3 + 2.25 * _absu(u) - 1.5,
)

KERNELS: Dict[str, Tuple[Callable, Callable, Callable, Callable, Callable, Callable]] = {
    "gauss": (K_gauss, K_gauss_p, K_gauss_pp, K2_gauss, K2_gauss_p, K2_gauss_pp),
    "epan" : (K_epan , K_epan_p , K_epan_pp , K2_epan , K2_epan_p , K2_epan_pp),
}

# ---------------------------------------------------------------------------
# 2.  Analytic LSCV score / gradient / Hessian  (generic kernel)
# ---------------------------------------------------------------------------

def lscv_generic(x: np.ndarray, h: float, kernel: str):
    """Return (LSCV, ∂, ∂²) at bandwidth *h* for the chosen kernel."""
    K, Kp, Kpp, K2, K2p, K2pp = KERNELS[kernel]
    n = len(x)
    u = (x[:, None] - x[None, :]) / h

    # ----- score -----
    term1 = K2(u).sum() / (n ** 2 * h)
    term2 = (K(u).sum() - np.sum(np.diag(K(u)))) / (n * (n - 1) * h)
    score = term1 - 2 * term2

    # ----- gradient -----
    S_F = (K2(u) + u * K2p(u)).sum()
    S_K = (K(u) + u * Kp(u)).sum() - 0.0  # i=j derivatives already excluded
    grad = -S_F / (n ** 2 * h ** 2) + 2 * S_K / (n * (n - 1) * h ** 2)

    # ----- Hessian -----
    S_F2 = (2 * K2p(u) + u * K2pp(u)).sum()
    S_K2 = (2 * Kp(u) + u * Kpp(u)).sum()
    hess = 2 * S_F / (n ** 2 * h ** 3) - S_F2 / (n ** 2 * h ** 2)
    hess += -4 * S_K / (n * (n - 1) * h ** 3) + 2 * S_K2 / (n * (n - 1) * h ** 2)
    return score, grad, hess

# Fast wrappers -------------------------------------------------------------
lscv_gauss = lambda x, h: lscv_generic(x, h, "gauss")
lscv_epan  = lambda x, h: lscv_generic(x, h, "epan")

# ---------------------------------------------------------------------------
# 3.  Newton–Armijo optimiser (scalar bandwidth)
# ---------------------------------------------------------------------------

def newton_opt(
    x: np.ndarray,
    h0: float,
    score_grad_hess,
    tol: float = 1e-5,
    max_iter: int = 12,
):
    h, evals = h0, 0
    for _ in range(max_iter):
        f, g, H = score_grad_hess(x, h); evals += 1
        if abs(g) < tol:
            break
        step = -g / H if (H > 0 and np.isfinite(H)) else -0.25 * g
        if abs(step) / h < 1e-3:
            break
        # Armijo back‑tracking (cheap, no new Jacobians)
        for _ in range(10):
            h_new = max(h + step, 1e-6)
            if score_grad_hess(x, h_new)[0] < f:
                h = h_new; break
            step *= 0.5
    return h, evals

# ---------------------------------------------------------------------------
# 4.  Other optimisers & helpers
# ---------------------------------------------------------------------------

def golden_section(obj, a: float, b: float, tol: float = 1e-3):
    gr = (np.sqrt(5) - 1) / 2
    c, d = b - gr * (b - a), a + gr * (b - a)
    fc, fd, evals = obj(c), obj(d), 2
    while b - a > tol:
        if fc < fd:
            b, d, fd = d, c, fc
            c = b - gr * (b - a)
            fc = obj(c)
        else:
            a, c, fc = c, d, fd
            d = a + gr * (b - a)
            fd = obj(d)
        evals += 1
    return (a + b) / 2, evals

def silverman_bandwidth(x: np.ndarray):
    n = len(x)
    sigma = np.std(x, ddof=1)
    iqr = np.subtract(*np.percentile(x, [75, 25]))
    sigma = min(sigma, iqr / 1.349)
    return 0.9 * sigma * n ** (-1 / 5)

# Newton initialised from coarse grid --------------------------------------

def bandwidth_newton(x: np.ndarray, kernel: str):
    grid = np.logspace(-1, 1, 20)
    scorefun = lscv_gauss if kernel == "gauss" else lscv_epan
    scores = [scorefun(x, h)[0] for h in grid]
    h0 = grid[int(np.argmin(scores))]
    return newton_opt(x, h0, scorefun)

# ---------------------------------------------------------------------------
# 5.  ISE benchmark utilities
# ---------------------------------------------------------------------------

def true_pdf(z: np.ndarray, noise: float):
    return 0.5 * norm.pdf(z, -2, np.sqrt(0.5 ** 2 + noise ** 2)) + 0.5 * norm.pdf(z, 2, np.sqrt(1.0 ** 2 + noise ** 2))

def ise_estimate(x: np.ndarray, h: float, noise: float, kernel: str):
    K, *_ = KERNELS[kernel]
    zz = np.linspace(-8, 8, 601)
    est = np.array([K((z - x) / h).mean() / h for z in zz])
    return np.trapz((est - true_pdf(zz, noise)) ** 2, zz)

# ---------------------------------------------------------------------------
# 6.  Monte‑Carlo benchmark (Gaussian & Epanechnikov)
# ---------------------------------------------------------------------------

def benchmark_kde(n: int, noise: float, kernel: str, R: int = 20):
    rng = np.random.default_rng(int(n * 123 + noise * 10))
    grid_h = np.logspace(-1, 1, 50)
    scorefun = lscv_gauss if kernel == "gauss" else lscv_epan
    methods = {m: ([], []) for m in ["grid", "golden", "newton", "silver"]}

    for _ in range(R):
        x = rng.normal(0, 1, n) + rng.normal(0, noise, n)

        # Grid ---------------------------------
        scores = [scorefun(x, h)[0] for h in grid_h]
        h_g = grid_h[int(np.argmin(scores))]
        methods["grid"][0].append(ise_estimate(x, h_g, noise, kernel))
        methods["grid"][1].append(len(grid_h))

        # Golden -------------------------------
        obj = lambda h: scorefun(x, h)[0]
        h_gold, eval_gold = golden_section(obj, grid_h[0], grid_h[-1])
        methods["golden"][0].append(ise_estimate(x, h_gold, noise, kernel))
        methods["golden"][1].append(eval_gold)

        # Newton -------------------------------
        h_new, eval_new = bandwidth_newton(x, kernel)
        methods["newton"][0].append(ise_estimate(x, h_new, noise, kernel))
        methods["newton"][1].append(eval_new)

        # Silverman ----------------------------
        h_sil = silverman_bandwidth(x)
        methods["silver"][0].append(ise_estimate(x, h_sil, noise, kernel))
        methods["silver"][1].append(1)

    return methods

# ---------------------------------------------------------------------------
# 7.  Pretty‑print helpers
# ---------------------------------------------------------------------------

def summarise(arr):
    return f"{np.mean(arr): .4f}±{np.std(arr): .4f}"

def summarise_int(arr):
    return f"{np.mean(arr): .1f}±{np.std(arr): .1f}"

# ---------------------------------------------------------------------------
# 8.  Main entry
# ---------------------------------------------------------------------------

def main():
    ns, noises = [100, 200, 500], [0.5, 1.0, 2.0]
    print("Method       n   noise  ker   ISE±SD        Evals±SD")
    for kernel in ["gauss", "epan"]:
        for n in ns:
            for noise in noises:
                res = benchmark_kde(n, noise, kernel, R=20)
                for m in ["grid", "golden", "newton", "silver"]:
                    ise_str = summarise(res[m][0]); ev_str = summarise_int(res[m][1])
                    print(f"{m:<10}{n:<6}{noise:<6}{kernel:<6}{ise_str:<15}{ev_str}")

if __name__ == "__main__":
    main()

Method       n   noise  ker   ISE±SD        Evals±SD
grid      100   0.5   gauss  0.1843± 0.0317 50.0± 0.0
golden    100   0.5   gauss  0.1818± 0.0308 22.0± 0.0
newton    100   0.5   gauss  0.1863± 0.0310 12.0± 0.0
silver    100   0.5   gauss  0.1906± 0.0233 1.0± 0.0
grid      100   1.0   gauss  0.0694± 0.0166 50.0± 0.0
golden    100   1.0   gauss  0.0674± 0.0119 22.0± 0.0
newton    100   1.0   gauss  0.0710± 0.0159 12.0± 0.0
silver    100   1.0   gauss  0.0750± 0.0132 1.0± 0.0
grid      100   2.0   gauss  0.0118± 0.0094 50.0± 0.0
golden    100   2.0   gauss  0.0118± 0.0094 22.0± 0.0
newton    100   2.0   gauss  0.0121± 0.0098 12.0± 0.0
silver    100   2.0   gauss  0.0115± 0.0051 1.0± 0.0
grid      200   0.5   gauss  0.1860± 0.0273 50.0± 0.0
golden    200   0.5   gauss  0.1846± 0.0252 22.0± 0.0
newton    200   0.5   gauss  0.1882± 0.0275 12.0± 0.0
silver    200   0.5   gauss  0.1889± 0.0229 1.0± 0.0
grid      200   1.0   gauss  0.0685± 0.0162 50.0± 0.0
golden    200   1.0   gauss  0.06